In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris

data = load_iris()

df_X = pd.DataFrame(data.data,columns=data.feature_names)
df_y = pd.DataFrame(data.target,columns=['species'])

df = pd.concat([df_X.iloc[:,0],df_y],axis=1)

speciesDict = {k : v for k, v in zip([0,1,2],data.target_names)} # 타켓으로 0,1,2 -> 타켓이름으로  
df['species'] = df['species'].replace(speciesDict) # 종속변수를 문자열로 변경  
df = df.sample(n=10,random_state=6)  # 임의 데이터 10개를 샘플링  
df.reset_index(drop=True, inplace=True) # 기존 인덱스 삭제하고 새롭게 인덱스 리셋


# 결손값이 대입 
df.iloc[-2:,:] = np.nan 

In [2]:
df

,sepal length (cm),species
0,5.0,setosa
1,6.5,virginica
2,4.7,setosa
3,5.1,setosa
4,6.3,virginica
5,5.7,versicolor
6,6.1,virginica
7,5.1,setosa
8,NaN,NaN
9,NaN,NaN


#### Q1 ).One-hot 인코딩 :변수의 라벨의 종류마다 특징량(열)을 생성하고, True(1), False(0)를 할당한다.

In [3]:
# pandas의 get_dummies를 사용하는 방법 :누락된 값은 인코딩되지 않음(모두 0이 됨)
df_dummies = pd.get_dummies(df['species'])
df_encoding = pd.concat([df,df_dummies],axis=1)
df_encoding

,sepal length (cm),species,setosa,versicolor,virginica
0,5.0,setosa,True,False,False
1,6.5,virginica,False,False,True
2,4.7,setosa,True,False,False
3,5.1,setosa,True,False,False
4,6.3,virginica,False,False,True
5,5.7,versicolor,False,True,False
6,6.1,virginica,False,False,True
7,5.1,setosa,True,False,False
8,NaN,NaN,False,False,False
9,NaN,NaN,False,False,False


In [5]:
#scikit-learn.preprocesssing을 사용하는 방법  : 결측값이 있으면 인코딩할 수 없기 때문에 대체 값이 필요한다.  
from sklearn.preprocessing import OneHotEncoder

import warnings
warnings.filterwarnings('ignore')

#결측값 보안  
df_temp = df.fillna('unknown')

#인코딩 : 원핫 인코딩에서는 필요한 것은  array  
ohe = OneHotEncoder(sparse=False) # 객체 생성   sparse=False :array  sparse=True:Matrix 
encoded = ohe.fit_transform(df_temp['species'].values.reshape(-1, 1))

#라벨의 이름을 리턴받고 접두어를 붙인다음 DataFrame로 생성한다. 
label = ohe.get_feature_names(['spcies'])
df_encoding = pd.DataFrame(encoded, columns=label, dtype=int)

df_encoding = pd.concat([df, df_encoding], axis=1)
df_encoding

AttributeError: 'OneHotEncoder' object has no attribute 'get_feature_names'

In [6]:
df_encoding

,sepal length (cm),species,setosa,versicolor,virginica
0,5.0,setosa,True,False,False
1,6.5,virginica,False,False,True
2,4.7,setosa,True,False,False
3,5.1,setosa,True,False,False
4,6.3,virginica,False,False,True
5,5.7,versicolor,False,True,False
6,6.1,virginica,False,False,True
7,5.1,setosa,True,False,False
8,NaN,NaN,False,False,False
9,NaN,NaN,False,False,False


#### Q2)  category_encoders를 사용하는 방법

In [5]:
#!pip install category_encoders

In [6]:
import category_encoders as ce
#handle_missing  : 'value(결측값도 변수 중 하나로 취급 )' 
#                : 'return_nan( 'NaN 채우기 ')'
#                : error   (인코딩하지 않고 error리턴  )

ce_ohe = ce.OneHotEncoder(cols=['species'],handle_missing='value')
df_encoding = ce_ohe.fit_transform(df)
df_encoding = pd.concat([df['species'],df_encoding],axis=1)
df_encoding

,species,sepal length (cm),species_1,species_2,species_3,species_4
0,setosa,5.0,1,0,0,0
1,virginica,6.5,0,1,0,0
2,setosa,4.7,1,0,0,0
3,setosa,5.1,1,0,0,0
4,virginica,6.3,0,1,0,0
5,versicolor,5.7,0,0,1,0
6,virginica,6.1,0,1,0,0
7,setosa,5.1,1,0,0,0
8,NaN,NaN,0,0,0,1
9,NaN,NaN,0,0,0,1


In [ ]:
#print(help(ce.OneHotEncoder))

#### Q3)  LabelEncoder를 사용하는 방법

In [7]:
# LabelEncoder : 결측값이 있으면 에라가 되므로 대체값이 줘야 한다.  
# case 1:   
# 각 라벨을 숫자로 변환, 숫자의 대소에는 의미가 없기 때문에, 회귀분석에 사용할 수 없다. 
# 결정 트리 기반 모델에 사용한다. 

from sklearn.preprocessing import LabelEncoder

#결측치 대입 
df_encoding = df.fillna('unknown')

#인코딩
le = LabelEncoder()
encoded = le.fit_transform(df_encoding['species'].values)# 종속변수 라벨인코드
df_encoding['label'] = encoded # 컬럼생성해서  대입 

# 디코더 : 원래 변수로 되돌린다.  
decoded = le.inverse_transform(encoded)
decoded

array(['setosa', 'virginica', 'setosa', 'setosa', 'virginica',
       'versicolor', 'virginica', 'setosa', 'unknown', 'unknown'],
      dtype=object)

In [ ]:
df_encoding

In [ ]:
#Case 2 :  
import category_encoders as ce

ce_oe = ce.ordinal.OrdinalEncoder(cols=['species'],handle_missing='value')
df_encoding = ce_oe.fit_transform(df)
df_encoding = pd.concat([df['species'],df_encoding],axis=1)
df_encoding

### Q4) 카운트 인코딩 : 데이터에 각 레벨의 카운트를 변수로 사용한다.  

In [ ]:
#pandas 메서드 (groupby)를 사용하는 방법 
#결측치 계산하지 않고 그대로 리턴  
df_encoding = df.copy()
df_encoding['count'] = df.groupby('species')['species'].transform('count')
df_encoding

In [ ]:
df

In [ ]:
#category_encoders
import category_encoders as ce
#  handle_missing='count'  : 누락된 값도 계산된다.  

ce_ce = ce.count.CountEncoder(cols=['species'],handle_missing='count')
df_encoding = ce_ce.fit_transform(df)
df_encoding = pd.concat([df['species'],df_encoding],axis=1)
df_encoding

### Q5)라벨 카운트 인코딩 : 카운트 랭킹

In [ ]:
#pandas groupby : 결측값은 카운트 순위기 지정되지 않고 그대로 리턴  
df_encoding = df.copy()
count_rank = df.groupby('species')['species'].count().rank(ascending=False)
df_encoding['countLabel'] = df['species'].map(count_rank)
df_encoding

In [ ]:
df_encoding.info()

### Q6)타겟 인코딩 
    1) https://proceedings.neurips.cc/paper/2018/file/14491b756b3a51daac41c24863285549-Paper.pdf 

    2) https://contrib.scikit-learn.org/category_encoders/index.html

각 라벨에 대해 목표 변수의 통계량을 계산하고 그값을 변수로 사용한다. 
TS(Target Statistics)를 이용하는 기법 
1) GreedyTS  : 기본 방법으로 라벨마다 통계량 계산  (비추천 ) 

2) HoldoutTs :  K-Fold의 CV로 데이터를 분할 하고 학습용 데이터를 사용하여 통계량 계산하고 홀드아웃 샘플의 변수로 사용한다. 

3) OrderedTs  :  우선순위를 지정하여 통계 라벨링  

4) Leave one- out Ts  : 자체(특징량을 부여하는 대상)   연산된 통계량을 제외한다.  (비추천) 

In [ ]:
#Case 1 : GreedyTS 
import category_encoders as ce

df_GreedyTS = df.copy()
te = ce.TargetEncoder(cols=['species'])
df_GreedyTS['GreedyTS']= te.fit_transform(df['species'], df['sepal length (cm)'])
df_GreedyTS

In [ ]:
# Case 2 :Leave one- out Ts 
df_LooTS = df.copy()
te = ce.LeaveOneOutEncoder(cols=['species'])
df_LooTS['LooTS']= te.fit_transform(df['species'], df['sepal length (cm)'])
df_LooTS

In [ ]:
#case 3: OrderedTs
df_OTS = df.copy()
te = ce.CatBoostEncoder(cols=['species'])
df_OTS['OrderedTS']= te.fit_transform(df['species'], df['sepal length (cm)'])
df_OTS